In [4]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from pandas_datareader.data import DataReader

import matplotlib.pylab as plt
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context('notebook', font_scale=1.2)

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing, Holt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA

import ipywidgets as wgts
from ipywidgets import interactive, IntSlider, ToggleButtons, FloatSlider

from sklearn.metrics import mean_squared_error

In [5]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [6]:
def plot_forecasts(y_train, y_test, y_pred, m):
    """
    """
    fig, ax = plt.subplots(figsize=(15, 5)) 
    y_train.plot(ax=ax, label='train')
    y_test.plot(ax=ax, label='actual')
    y_pred.plot(ax=ax, label='predicted')
    m.fittedvalues.plot(ax=ax, alpha=0.5, label='fitted', style='--')
    plt.legend(loc='best')
    
    print(f"""\n\nMetrics -> 
    BIC = {m.bic}, 
    AIC = {m.aic}, 
    RMSE = {np.sqrt(mean_squared_error(y_test, y_pred)).round(2)},
    MAPE = {mean_absolute_percentage_error(y_test, y_pred).round(2)}
    \n""")
    ;

def plot_residuals(y_test, y_pred):
    """
    """
    delta = y_test - y_pred
    fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(15, 4)) 
    delta.plot(ax=axs[0], title='Residuals', style='k--', alpha=0.6)
    delta.plot.hist(ax=axs[1], bins=20, title='Distribution', color='k', alpha=0.5);

In [85]:
def format_plot(ax):
    """
    """
    font_title = {
        'size': 20, 
        'weight': 600, 
        'name': 'monospace'
    }

    font_axes = {
        'size': 14, 
        'weight': 'bold', 
        'name': 'monospace'
    }

    ax.grid(True, linestyle=":", alpha=0.6)
    sns.despine(ax=ax, left=True)
    
    if ax.get_legend():
        ax.legend(bbox_to_anchor=(1.1, 1))
    
    ax.set_title(f"\n{ax.get_title()}\n", fontdict=font_title)
    ax.set_xlabel(f"\n{ax.get_xlabel()} ➞", fontdict=font_axes)
    ax.set_ylabel(f"{ax.get_ylabel()} ➞\n", fontdict=font_axes)

## Simulator 1: Creates ARIMA Series given AR, MA coeffs

In [387]:
def simulate_ARIMA_from_coeffs(AR: np.array, I_: int, MA: np.array, n_samples: int) -> np.array:
    """
    """
    ar_ = np.r_[1, -np.array(AR)]
    ma_ = np.r_[1, np.array(MA)]

    y = sm.tsa.arma_generate_sample(ar=ar_,
                                    ma=ma_,
                                    nsample=n_samples,
                                    distrvs=np.random.RandomState(42).randn)

    if I_ > 0:
        for i in range(I_):
            y = y.cumsum()
    
    return y

In [396]:
def f(AR, I_, MA):
    """
    """
    y = simulate_ARIMA_from_coeffs(AR=AR if AR[0] > 0 else [], 
                                   I_=I_, 
                                   MA=MA if MA[0] > 0 else [], 
                                   n_samples=120)
    
    s_ = pd.Series(data=y, index=pd.date_range(start='1990-01-01', 
                                                freq='MS', 
                                                periods=len(y)))
    
    title_=f"""
    ARIMA({len(AR) if AR[0] > 0 else 0}, {I_}, {len(MA) if MA[0] > 0 else 0}),
    AR Parameters: {[round(i, 3) for i in AR if AR[0] is not None]}
    Differenced: {I_}
    MA Parameters: {[round(i, 3) for i in MA if MA[0] is not None]}
    """
       
    ax = s_.plot(figsize=(15, 7), title=title_)    
    format_plot(ax)
    ax.set_ylabel('y')
    ax.set_xlabel('time')
    return None
    
interactive(f, 
            AR = wgts.SelectMultiple(options=np.arange(0, 1, 0.1).round(2).tolist(), 
                                     index=[1],
                                     description='AR Coeffs'),
            I_ = IntSlider(min=0, max=2, step=1, description='Difference Degree'),
            MA = wgts.SelectMultiple(options=np.arange(0, 1, 0.1).round(2).tolist(), 
                                     index=[1],
                                     description='MA Coeffs'))    

interactive(children=(SelectMultiple(description='AR Coeffs', index=(1,), options=(0.0, 0.1, 0.2, 0.3, 0.4, 0.…

## Simulator 1: Creates random ARIMA Series given p, d, q

In [418]:
def simulate_ARIMA_from_pqd(p, d, q):
    """
    """
    AR_ = [] if p == 0 else Series(np.random.random(p).round(2)).sort_values().values
    MA_ = [] if q == 0 else Series(np.random.random(q).round(2)).sort_values().values
    
    y = simulate_ARIMA_from_coeffs(AR=AR_, 
                                   I_=d, 
                                   MA=MA_, 
                                   n_samples=250)
    if d > 0:
        for i in range(d):
            y = y.cumsum()
    
    str_ar = f"AR=[{','.join(AR_.astype(str))}]" if p > 0 else ''
    str_ma = f"MA=[{','.join(MA_.astype(str))}]" if q > 0 else ''
    name_ = '_'.join([str_ar, str_ma])
    
    return Series(y, name=name_, index=pd.date_range(start='1990-01-01', 
                                                     freq='MS', 
                                                     periods=len(y)))

In [420]:
def f(pdq):
    """
    """
    p, d, q = [int(i) for i in pdq[0].split(",")]
    df_ = pd.concat([simulate_ARIMA_from_pqd(p, d, q) for i in range(10)], axis=1).round(2)
    ax_ = df_.plot(figsize=(15, 7), 
                   alpha=0.5, 
                   title=f"ARIMA({pdq[0]})")
    format_plot(ax_)
    
interactive(
    f,
    pdq=wgts.SelectMultiple(options=[f"{ar},{i},{ma}" for ar in range(2) for i in range(3) for ma in range(3)], 
                          value=['1,0,1'], 
                          description='ARIMA order')
)


interactive(children=(SelectMultiple(description='ARIMA order', index=(10,), options=('0,0,0', '0,0,1', '0,0,2…